# Associated Sports Events Column
This Jupyter Notebook will be used to parse the webpages and add data to the column in the master dataset which mentions related sports event based on date

In [51]:
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import csv
import warnings
warnings.filterwarnings('ignore')

Here we define a few functions, parse_link parses the webpages for the sports events tables, and saves those tables in the raw_files folder. The parse_dates function parses the string dates and converts them into date time objects showing the start and end datetime of a particular sports event. Finally, the save_formatted_sports_events_table brings both of these functions together and saves the formatted version.

In [52]:
def parse_link(url, year):
    # make a request to the webpage
    response = requests.get(url)

    # parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the table containing the events
    table = soup.find('table', {'class': 'list'})

    # find all the rows in the table
    rows = table.find_all('tr')

    # create a CSV file to write the scraped data
    with open(f'raw_files/events_{year}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # write the table headers to the CSV file
        headers = [th.text.strip() for th in rows[0].find_all('th')]
        writer.writerow(headers)

        # write the table rows to the CSV file, filtering out rows containing "postponed" or "canceled"
        for row in rows[1:]:
            data = [td.text.strip() for td in row.find_all('td')]
            data = [x.replace('posponed', "postponed") for x in data]
            data = [x.replace('postoned', "postponed") for x in data]

            if 'postponed from' in data[0].lower():
                writer.writerow(data)
            elif 'canceled' in data[0].lower() or 'postponed' in data[0].lower():
                continue
            else:
                writer.writerow(data)


def parse_dates(date_str, year):
    date_str = date_str.split(' (')[0]
    date_str = date_str.replace('July', 'Jul')
    date_str = date_str.replace('June', 'Jun')
    date_str = date_str.replace('Sept', 'Sep')
    date_str = date_str.replace('–', '-')
    date_str = date_str.replace('8 Mar', 'Mar 8')
    date_str = date_str.replace('1 Mar', 'Mar 1')

    # split date range into start and end dates
    date_range = date_str.split('-')

    date_range = [x.strip() for x in date_range]

    if len(date_range) == 1:
        # if there is only one date, it is both the start and end date
        date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
        date_end = date_start
    else:
        # if second value can be converted to int, this is an individual month
        if len(date_range[-1]) <= 2:
            date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
            date_end = datetime(int(year), date_start.month, int(date_range[-1]))
        else:
            # otherwise, parse both start and end dates
            date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
            date_end = datetime.strptime(date_range[1] + f' {year}', '%b %d %Y')

    return date_start, date_end

def save_formatted_sports_events_table(url, year):
    parse_link(url, year)
    save_filename = f"raw_files/events_{year}.csv"
    df = pd.read_csv(save_filename)
    df['date_start'], df['date_end'] = zip(*df['Date(s)'].apply(parse_dates, year=year))
    df.to_csv(save_filename, index=False)

Now that we have the functions, we iterate over the years 2020, 2021, and 2022 and save the tables. We then merge these tables into one df, and save that csv.

In [53]:
years = [2020, 2021, 2022]
dfs = list()

for year in years:
    url = f'https://www.topendsports.com/events/calendar-{year}.htm'
    save_formatted_sports_events_table(url, year)

for year in years:
    df = pd.read_csv(f"raw_files/events_{year}.csv")
    dfs.append(df)

total = pd.concat(dfs)
total.drop(columns='Date(s)', inplace=True)
total.to_csv('raw_files/sports_events.csv', index=False)

We now have a fully merged csv with everything we need. We need to convert this into a df which has dates ranging from Jan 1st 2020 to Dec 31st 2022, which we will further merge with our master dataset.

In [54]:
df = pd.read_csv("raw_files/sports_events.csv")
# convert start and end times to datetime
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

# create a range of dates from Jan 1, 2020 to Dec 31, 2022
dates = pd.date_range(start='2020-01-01', end='2022-12-31')

# create an empty dataframe to store the results
result = pd.DataFrame(columns=['Date', 'Event'])

# iterate over each date and find the events that occurred on that date
for date in dates:
    events = df.loc[(df['date_start'] <= date) & (df['date_end'] >= date), 'Event'].tolist()
    event_str = ', '.join(events)
    result = result.append({'Date': date, 'Event': event_str}, ignore_index=True)

# print the resulting dataframe
result.to_csv("sports_events_final.csv", index=False)

We got our df to merge. We finally perform the merging.

In [55]:
df1 = pd.read_csv('../Master_Dataset_Raw.csv')
df2 = pd.read_csv("sports_events_final.csv")

og_dates = df1['Account Created Date']

# Convert "Account Created Date" column in df1 to datetime
df1['Account Created Date'] = pd.to_datetime(df1['Account Created Date'])

# Convert "Date" column in df2 to datetime
df2['Date'] = pd.to_datetime(df2['Date'])

# merge the two dataframes based on the dates
merged_df = pd.merge(df1, df2[['Date', 'Event']], how='left', left_on=df1['Account Created Date'].dt.date, right_on=df2['Date'].dt.date)

# drop the duplicate date column
merged_df = merged_df.drop('key_0', axis=1)

# rename the 'Event' column to 'Events'
merged_df = merged_df.rename(columns={'Event': 'Events'})

# set the 'Events' column to NaN where there is no matching date
merged_df.loc[merged_df['Events'].isna(), 'Events'] = ''

# drop the duplicate date column
merged_df = merged_df.drop('Date', axis=1)

merged_df = merged_df.rename(columns={'Events': 'Sports Events'})

merged_df.to_csv('Master_Dataset_Raw_SportsEvents.csv', index=False)

# display the resulting dataframe
merged_df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest,Sports Events
0,156581,STY1666112066,86625,USR1662300553,NaN,12,Qatar Airways cuts flights to accommodate FIFA...,"<p dir=""ltr"">Qatar Airways&#160;has withdrawn ...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 19:39:13,"Technology, History, Food, Entertainment, Spor...","World Volleyball Championships (men), US Open"
1,156580,STY1666111954,119236,USR1665943314,NaN,18,Positive thinking does not necessarily mean av...,"<p dir=""ltr"">Positive thinking does not necess...",https://image.pixstory.com/Pixstory-image-1666...,2022-10-16 23:31:54,"Technology, History, Food, Entertainment, Spor...","Womens World Cup, World Track Championships, W..."
2,156579,STY1666111923,86566,USR1662294899,NaN,12,FIFA negotiating 'initiatives' for Qatar migra...,"<p dir=""ltr"">FIFA's president sought Monday to...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 18:04:59,"Technology, History, Food, Entertainment, Spor...","World Volleyball Championships (men), US Open"
3,156578,STY1666111798,86655,USR1662303432,NaN,12,Qatar ready for the World Cup in just over a m...,"<p dir=""ltr"">Ticket sales for the FIFA World C...",https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 20:27:12,"Technology, History, Food, Entertainment, Spor...","World Volleyball Championships (men), US Open"
4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","<p dir=""ltr""></p>\n<p dir=""ltr"">Almost all of ...",https://image.pixstory.com/Pixstory-image-1666...,2021-09-27 13:00:15,"Science, Inequality, Education, Technology, He...",
...,...,...,...,...,...,...,...,...,...,...,...,...
94995,95021,STY1650382324,22770,USR1642425119,male,40,Ronaldo salta la trasferta di Liverpool,Il Manchester United ha annunciato l'assenza d...,https://image.pixstory.com/Pixstory-image-1650...,2022-01-17 18:41:59,"calcio, premier league, cristiano ronaldo, man...","Africa Cup of Nations, Australian Open"
94996,95020,STY1650381837,32577,USR1650381467,NaN,15,Name-vishal\nFather name-Amod\nClass-7\nAge-13...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:47:47,chintan,"Invictus Games, World Snooker Championship"
94997,95019,STY1650381574,32498,USR1650370047,NaN,15,Earth day,Name Muskan \nClass 8\nSex f,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 17:37:27,chintan,"Invictus Games, World Snooker Championship"
94998,95018,STY1650380690,32574,USR1650380153,NaN,12,Name-puneet\nclass-6th\nfather name-Ramkishan\...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:25:53,chintan,"Invictus Games, World Snooker Championship"
